In [40]:
#Imporing the libraries
import os
import pandas as pd
import numpy as np
import statistics as stat

In [41]:
#Load the ratings dataset
ratings_data = pd.read_csv(r'D:/MS/Recommender Systems/ml-100k/u.csv',sep='\t', header=None)
# Understanding the Dataset for Ratings
ratings_data.columns=['User_ID','Item_ID','Rating','Timestamp']
ratings_data.head()

,User_ID,Item_ID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [42]:
# Looking at the Dataset--> Timestamp can be removed as it is not necessary for processing
ratings_data.drop('Timestamp', inplace=True, axis=1)
# Inspecting Data after dropping the 'timestamp' column
ratings_data.head()

,User_ID,Item_ID,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [43]:
# Length of the rows
print(len(ratings_data))
# Number of unique users
print(len(pd.unique(ratings_data['User_ID'])))

100000
943


In [44]:
d = 'Item_ID | movie_title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
#column_names2
# Loading the movies dataset
movies_data = pd.read_csv(r'D:/MS/Recommender Systems/ml-100k/uitem.csv', sep='|',header=None,names=column_names2,encoding='latin-1')
# Understanding the Dataset for Movies Dataset
movies_data.head()

,Item_ID,movie_title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [45]:
# Getting all the movies of a selected user has seen and rated
# This will work as an input values and these values will be used to recommend this user some new movies
# Selected userId (405) you can use any value
uId = 405
inputMovies = ratings_data[ratings_data['User_ID']== uId]
inputMovies.head()
# len(inputMovies)

,User_ID,Item_ID,Rating
12276,405,56,4
12383,405,592,1
12430,405,1582,1
12449,405,171,1
12460,405,580,1


In [47]:
# Number of movies that selected user has rated/watched
print(len(inputMovies))
# Now we will find out the similar users like our selected user
# Alternately, we can say, finding the users who rates similar movies as our selected user
similarUsers = ratings_data[ratings_data['Item_ID'].isin(inputMovies['Item_ID'].tolist())]
similarUsers
print(similarUsers['User_ID'].value_counts())

737
405    737
846    346
13     335
655    327
450    318
      ... 
529      1
657      1
302      1
817      1
155      1
Name: User_ID, Length: 930, dtype: int64


In [48]:
# Now creating sub dataframes based on userId
similarUsersGroup = similarUsers.groupby(['User_ID'])
# Inspecting one random user, let's say userId = 450
similarUsersGroup.get_group(450)
# len(similarUsersGroup)

,User_ID,Item_ID,Rating
17656,450,470,5
17680,450,783,3
17764,450,1147,4
18055,450,58,3
18071,450,142,5
...,...,...,...
98389,450,178,4
98566,450,584,5
98871,450,732,3
99039,450,388,3


In [49]:
# Let's sort those users dataframes
# So that, users with most in common with the selected user will have the priority
similarUsersGroup = sorted(similarUsersGroup, key= lambda x:len(x[1]), reverse = True)
# Inspecting the 2nd top user (1st one is the same as our selected user)
similarUsersGroup[1]

(846,
        User_ID  Item_ID  Rating
 56664      846     1074       3
 56764      846       94       4
 56811      846      627       4
 56859      846       57       2
 57066      846      377       2
 ...        ...      ...     ...
 99238      846      210       5
 99549      846      318       5
 99669      846       40       2
 99670      846       98       4
 99809      846      101       4
 
 [346 rows x 3 columns])

In [50]:
# Number of similar users, who rated all the movies or some of them.
print(len(similarUsersGroup))
# userId of the 2nd top user group
print(similarUsersGroup[1][0])
# Dataframe of the 2nd top user group
print(similarUsersGroup[1][1])

930
846
       User_ID  Item_ID  Rating
56664      846     1074       3
56764      846       94       4
56811      846      627       4
56859      846       57       2
57066      846      377       2
...        ...      ...     ...
99238      846      210       5
99549      846      318       5
99669      846       40       2
99670      846       98       4
99809      846      101       4

[346 rows x 3 columns]


In [11]:
#let's check how many movies this user has rated
len(similarUsersGroup[1][1])

346

In [51]:
# Number of similar users is very high (346) and not all the similar users are not contributing well for the recommendation
# Here, limiting the number to 100 Similar Users while computing the Similarity Score using pearson corrrelation.

# similarUsersGroup = similarUsersGroup[1:101] # Removing the first user (userId =405), as it is our selected user
similarUsersGroup = similarUsersGroup[1:101]
len(similarUsersGroup)
print(similarUsersGroup)

[(846,        User_ID  Item_ID  Rating
56664      846     1074       3
56764      846       94       4
56811      846      627       4
56859      846       57       2
57066      846      377       2
...        ...      ...     ...
99238      846      210       5
99549      846      318       5
99669      846       40       2
99670      846       98       4
99809      846      101       4

[346 rows x 3 columns]), (13,        User_ID  Item_ID  Rating
63          13      526       3
185         13       98       4
304         13      198       3
321         13      215       5
356         13       56       5
...        ...      ...     ...
97455       13      400       4
97858       13      541       1
99170       13      432       4
99306       13      904       1
99801       13      446       1

[335 rows x 3 columns]), (655,        User_ID  Item_ID  Rating
39576      655      693       3
39641      655      198       4
39693      655      670       3
39780      655      523       3
39

In [52]:
#Store the Pearson Correlation in a dictionary, 
# where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every similar user group in our subset
for name, group in similarUsersGroup:
    
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='Item_ID')
    inputMovies = inputMovies.sort_values(by='Item_ID')
    
    #Get the N (total similar movies watched) for the formula 
    #nRatings = len(group)
    
    #Get the review scores of the Selected user for the movies that they both have in common
    temp_df = inputMovies[inputMovies['Item_ID'].isin(group['Item_ID'].tolist())]
    
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    selectedUserTempRatingList = temp_df['Rating'].tolist()
    #print(selectedUserTempRatingList)
    
    #Let's also put the current user group reviews in a list format
    similarUserTempRatingList = group['Rating'].tolist()
    
    #Calculating pearson similarity
    simXX = 0
    meanSelectedUserTempRating = stat.mean(selectedUserTempRatingList)
    for i in selectedUserTempRatingList:
        simXX = simXX + pow((i - meanSelectedUserTempRating),2)
    
    simYY = 0
    meanSimilarUserTempRaning = stat.mean(similarUserTempRatingList)
    for j in similarUserTempRatingList:
        simYY = simYY + pow((j - meanSimilarUserTempRaning),2)
        
    simXY = 0
    for i, j in zip(selectedUserTempRatingList, similarUserTempRatingList):
        simXY = simXY+ ((i - meanSelectedUserTempRating ) * (j-meanSimilarUserTempRaning))
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if simXX != 0 and simYY != 0:
        pearsonCorrelationDict[name] = simXY/np.sqrt(simXX*simYY)
    else:
        pearsonCorrelationDict[name] = 0                    

In [53]:
pearsonCorrelationDict.items()

dict_items([(846, 0.17010850832588198), (13, 0.16571860162076085), (655, 0.030335053411754367), (450, 0.1351280374070967), (276, 0.3197632949832927), (537, 0.07613540816499086), (7, 0.21510987177036997), (94, 0.094500745481761), (303, 0.31890705970857397), (59, 0.16625616668161414), (429, 0.2472260089383148), (234, 0.033177612850703), (416, 0.2663795737832617), (393, 0.1503928783016631), (222, 0.3429066839146551), (308, 0.008433619745827807), (682, 0.1652429800578542), (293, 0.1322471791189863), (378, 0.20435360376779346), (201, 0.16931860454981412), (279, -0.0875005382865721), (311, 0.07994191300292676), (561, -0.08846861268230952), (896, 0.3067435633859717), (417, 0.24120007956206244), (92, 0.09402816324682843), (796, 0.1847371600144272), (194, 0.19668865053987647), (435, 0.03660113145465196), (406, -0.008587315975503869), (95, 0.2650805900462755), (642, 0.044018932181595874), (387, 0.019417754843104904), (399, 0.004598436073649535), (268, 0.09246794548934029), (804, 0.14833554594399

In [54]:
similarityScoreofSimilarUsers_data = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
similarityScoreofSimilarUsers_data.head()
similarityScoreofSimilarUsers_data[0][0:10] ###Similar Users for user 405

846    0.170109
13     0.165719
655    0.030335
450    0.135128
276    0.319763
537    0.076135
7      0.215110
94     0.094501
303    0.318907
59     0.166256
Name: 0, dtype: float64

# Using the Prediction Function to predict the Rating of a particular movie 

In [55]:
### Using the prediction function given:
# User_ID=450 is A
# User_ID=846 is B
# similarUsersGroup[0][846]
similarUsers = similarUsers.groupby(['User_ID'])
userA=similarUsers.get_group(450)
userB=similarUsers.get_group(846)

# Get the movies user B which are not rated by User A
movieIDNotinUserA=userB[~userB['Item_ID'].isin(userA['Item_ID'])]
len(movieIDNotinUserA)


118

In [56]:
movieIDNotinUserA

,User_ID,Item_ID,Rating
56664,846,1074,3
56859,846,57,2
57066,846,377,2
58372,846,36,2
58830,846,674,4
...,...,...,...
97361,846,738,4
98580,846,1109,3
99105,846,1178,2
99211,846,429,2


In [57]:
# calculate mean ratings for Users A and B
meanA=userA['Rating'].mean()
print(meanA)

meanB=userB['Rating'].mean()
print(meanB)

similarityAB=similarityScoreofSimilarUsers_data[0][846] ###Similarity between Users 450 and 846

# Here the UserB is 846 and the item which is present in B and not in A is 1074.
# Hence calculating prediction for User A item 1074
userBItem=userB[userB['Item_ID'] == 1074]
ratingsUserBItem=userBItem['Rating']

#Predicting the Rating for User A and Movie_ID 1074
predictionAB= meanA + ((similarityAB * (ratingsUserBItem-meanB))/similarityAB)

list(predictionAB)

3.8867924528301887
3.6878612716763004


[3.1989311811538883]

# 10 Similar Users for the Selected User(405) 

In [58]:
#Similarity score of other similar users to our selected user (405)
similarityScoreofSimilarUsers_data.columns = ['Similarity_Score']
similarityScoreofSimilarUsers_data['userId'] = similarityScoreofSimilarUsers_data.index
similarityScoreofSimilarUsers_data.index = range(len(similarityScoreofSimilarUsers_data))
similarityScoreofSimilarUsers_data.head()
# type(similarityScoreofSimilarUsers_data)

,Similarity_Score,userId
0,0.170109,846
1,0.165719,13
2,0.030335,655
3,0.135128,450
4,0.319763,276


In [60]:
topSimilarUsers=similarityScoreofSimilarUsers_data.sort_values(by='Similarity_Score', ascending=False)
topSimilarUsers.head(10)

,Similarity_Score,userId
57,0.379618,130
14,0.342907,222
4,0.319763,276
8,0.318907,303
38,0.318099,880
95,0.312122,618
23,0.306744,896
43,0.299486,864
52,0.287451,749
92,0.285906,622


# 20 most relevant movies recommender suggests

In [61]:
# Merging the top similar users similarity score with their ratings of movies
topSimilarUsersRating = topSimilarUsers.merge(ratings_data, left_on='userId', right_on='User_ID', how='inner')
topSimilarUsersRating.head()

,Similarity_Score,userId,User_ID,Item_ID,Rating
0,0.379618,130,130,379,4
1,0.379618,130,130,216,4
2,0.379618,130,130,1014,3
3,0.379618,130,130,1017,3
4,0.379618,130,130,109,3


In [62]:
# Calculating the mean ratings of all the similar users
meanRb = topSimilarUsersRating.groupby('userId').mean()[['Rating']]
meanRb.columns = ['Avg_Rating']
meanRb['userId'] = meanRb.index
meanRb.index = range(len(meanRb))
meanRb

,Avg_Rating,userId
0,3.610294,1
1,3.965261,7
2,3.097484,13
3,3.880866,18
4,3.934555,59
...,...,...
95,3.362500,886
96,3.435583,889
97,4.000000,892
98,2.980663,896


In [63]:
# Merging the top similar users average rating score 
topSimilarUsersRating = topSimilarUsersRating.merge(meanRb, left_on='userId', right_on='userId', how='inner')
topSimilarUsersRating.head()

,Similarity_Score,userId,User_ID,Item_ID,Rating,Avg_Rating
0,0.379618,130,130,379,4,3.985836
1,0.379618,130,130,216,4,3.985836
2,0.379618,130,130,1014,3,3.985836
3,0.379618,130,130,1017,3,3.985836
4,0.379618,130,130,109,3,3.985836


In [64]:
#Multiplies the similarity by the user's ratings
topSimilarUsersRating['Weighted_Rating_Score'] = topSimilarUsersRating['Similarity_Score']*(topSimilarUsersRating['Rating']-topSimilarUsersRating['Avg_Rating'])
topSimilarUsersRating.head()

,Similarity_Score,userId,User_ID,Item_ID,Rating,Avg_Rating,Weighted_Rating_Score
0,0.379618,130,130,379,4,3.985836,0.005377
1,0.379618,130,130,216,4,3.985836,0.005377
2,0.379618,130,130,1014,3,3.985836,-0.374241
3,0.379618,130,130,1017,3,3.985836,-0.374241
4,0.379618,130,130,109,3,3.985836,-0.374241


In [66]:
#Applies a sum to the topUsers after grouping it up by userId
###???
tempTopSimilarUsersRating = topSimilarUsersRating.groupby('Item_ID').sum()[['Weighted_Rating_Score']]
tempTopSimilarUsersRating.columns = ['Sum_Weighted_Rating_Score']
tempTopSimilarUsersRating['Movie_ID'] = tempTopSimilarUsersRating.index
tempTopSimilarUsersRating.head()

,Sum_Weighted_Rating_Score,Movie_ID
Item_ID,,
1,6.259708,1
2,-2.423070,2
3,-3.363739,3
4,1.940318,4
5,-1.279583,5


In [67]:
#Creates an empty dataframe
recommendation_data = pd.DataFrame()
#Now we take the weighted average
meanRa = inputMovies['Rating'].mean()
recommendation_data['Weighted_Avg_Recommendation_Score'] = meanRa+(tempTopSimilarUsersRating['Sum_Weighted_Rating_Score']/topSimilarUsers['Similarity_Score'].sum())
recommendation_data['Movie_ID'] = recommendation_data.index
recommendation_data.head(20)


,Weighted_Avg_Recommendation_Score,Movie_ID
Item_ID,,
1,2.330803,1
2,1.642336,2
3,1.567749,3
4,1.988314,4
5,1.733004,5
6,1.792902,6
7,2.268328,7
8,2.260837,8
9,2.228547,9


In [68]:
recommendation_data_sorted=recommendation_data.sort_values(by='Weighted_Avg_Recommendation_Score', ascending=False)
recommendation_data_sorted.head(20)

,Weighted_Avg_Recommendation_Score,Movie_ID
Item_ID,,
50,3.085856,50
174,3.010456,174
172,2.965200,172
64,2.857734,64
98,2.830892,98
181,2.801748,181
12,2.756826,12
127,2.731610,127
173,2.727707,173


In [69]:
movies=movies_data[['Item_ID','movie_title']]
movies

,Item_ID,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [70]:
###Top 20 movie recommendations for the selected user(405)
top_20_movies=pd.merge(recommendation_data_sorted,movies, how='inner', on='Item_ID')
top_20_movies.head(20)

,Item_ID,Weighted_Avg_Recommendation_Score,Movie_ID,movie_title
0,50,3.085856,50,Star Wars (1977)
1,174,3.010456,174,Raiders of the Lost Ark (1981)
2,172,2.965200,172,"Empire Strikes Back, The (1980)"
3,64,2.857734,64,"Shawshank Redemption, The (1994)"
4,98,2.830892,98,"Silence of the Lambs, The (1991)"
5,181,2.801748,181,Return of the Jedi (1983)
6,12,2.756826,12,"Usual Suspects, The (1995)"
7,127,2.731610,127,"Godfather, The (1972)"
8,173,2.727707,173,"Princess Bride, The (1987)"
9,22,2.695014,22,Braveheart (1995)
